General Setup for all the datasets( first 3 cells)

In [6]:
import pyterrier as pt
from experiment_utils.experiments_helper import time_fct

if not pt.started():
    pt.init()

path_to_root = "../../"

PyTerrier 0.10.0 has loaded Terrier 5.9 (built by craigm on 2024-05-02 17:40) and terrier-helper 0.0.8

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


Evaluation metrics used for all the datasets

In [7]:
from pyterrier.measures import RR, nDCG, MAP

eval_metrics = [RR @ 10, nDCG @ 10, MAP @ 100]

Create the query encoder that will run on CPU. Encoder used for embedding all the datasets/queries

In [8]:
from encoders.snowflake_arctic_embed_m import SnowFlakeQueryEncoder

package = "Snowflake/"
model_name = "snowflake-arctic-embed-m"
q_encoder = SnowFlakeQueryEncoder(package + model_name)


## NFCorpus

In the following experiments, the alpha tuning is not used and we preset it to 0.05. We only wanted to see how fast each experiment. Timeit library was not used because I wanted to first understand if these are normal running times overall and then assess the latency for each experiment.

Sparse Retrieval

In [9]:
from experiment_utils.experiments_helper import test_first_stage_retrieval_name

dataset_name = "nfcorpus"
# dev_set_name = "irds:beir/nfcorpus/dev"
test_set_name = "irds:beir/nfcorpus/test"

pipeline_name = "BM25"

time_fct(test_first_stage_retrieval_name, dataset_name, test_set_name, eval_metrics, pipeline_name, path_to_root)

Experiment took 3.236 seconds to execute.


,name,RR@10,nDCG@10,AP@100
0,nfcorpus: BM25,0.534378,0.322219,0.143582


FFI Pipeline

In [12]:
from experiment_utils.experiments_helper import default_complete_test_pipeline_name

pipeline_name = "BM25 >> " + model_name
time_fct(default_complete_test_pipeline_name, dataset_name, test_set_name,
         q_encoder, eval_metrics, model_name, pipeline_name, path_to_root)

No third element available


TypeError: can only concatenate str (not "NoneType") to str

Run pipeline for FIQA dataset

In [29]:
from experiment_utils.experiments_helper import default_complete_test_pipeline_name

dataset_name = "fiqa"
# dev_set_name = "irds:beir/fiqa/dev"
dataset_test_name = "irds:beir/fiqa/test"

time_fct(
    default_complete_test_pipeline_name, dataset_name, dataset_test_name, q_encoder, eval_metrics)



100%|██████████| 57638/57638 [00:00<00:00, 822787.06it/s]


Experiment took 56.576 seconds to execute.


,name,RR@10,nDCG@10,AP@100
0,fiqa: BM25 >> gte-base-en-v1.5,0.399449,0.335966,0.274557


## For the Scidocs dataset, considering the lack of a dev set, the train set was used for finetuning the alpha value.

Given that the scidocs dataset offers only one dataset, we will split it into dev and test set. More exactly, we will split the topics because that is what we are testing against. I chose the 'text' topics as this dataset offers more topics categories.

In [5]:
from experiment_utils.experiments_helper import split_dev_test, default_complete_test_pipeline

dataset_name = "scidocs"
dataset = pt.get_dataset("irds:beir/scidocs")
topics = dataset.get_topics('text')

dev_topics, test_topics = split_dev_test(topics, test_size=0.8)

time_fct(default_complete_test_pipeline, dataset_name, dataset.get_qrels(), test_topics, q_encoder,
         eval_metrics)


100%|██████████| 25657/25657 [00:00<00:00, 679372.34it/s]


Experiment took 76.129 seconds to execute.


,name,RR@10,nDCG@10,AP@100
0,scidocs: BM25 >> gte-base-en-v1.5,0.292146,0.165924,0.113075


A similar approach is also followed for the "cqadupstack/english" dataset.

In [4]:
from experiment_utils.experiments_helper import split_dev_test, default_complete_test_pipeline

dataset_name = "cqadupstack/english"
dataset = pt.get_dataset("irds:beir/cqadupstack/english")
topics = dataset.get_topics('text')

dev_topics, test_topics = split_dev_test(topics, test_size=0.8)

time_fct(default_complete_test_pipeline, dataset_name, dataset.get_qrels(), test_topics, q_encoder, eval_metrics)


100%|██████████| 40221/40221 [00:00<00:00, 1227563.21it/s]


Experiment took 153.739 seconds to execute.


,name,RR@10,nDCG@10,AP@100
0,cqadupstack/english: BM25 >> gte-base-en-v1.5,0.366865,0.356954,0.326041


A similar approach is also followed for the "arguana" dataset.

### DelftBlue runtime : 15 minutes. Local runtime : 20 minutes

In [10]:
from experiment_utils.experiments_helper import split_dev_test, default_complete_test_pipeline

dataset_name = "arguana"
dataset = pt.get_dataset("irds:beir/arguana")
topics = dataset.get_topics()

dev_topics, test_topics = split_dev_test(topics, test_size=0.8)

time_fct(default_complete_test_pipeline, dataset_name, dataset.get_qrels(), test_topics, q_encoder,
         eval_metrics)



100%|██████████| 8674/8674 [00:00<00:00, 851624.37it/s]


Experiment took 1163.829 seconds to execute.


,name,RR@10,nDCG@10,AP@100
0,arguana: BM25 >> gte-base-en-v1.5,0.252108,0.376144,0.262613


Will rerun this cell after reindexing the dense index as there is a problem with some documents not being found( no vectors for...). Is it possible that I made the dense index correctly and the "irds:beir/scifact/test" misses some documents that are tested in "irds:beir/scifact". The error is also reproduced in the debug.ipynb where it can be observed that using only the sparse index does not cause any error so for that reason I think it is because of the dense one.

In [4]:
from experiment_utils.experiments_helper import default_complete_test_pipeline_name

dataset_name = "scifact"
# dev_set_name = "irds:beir/scifact/train"
dataset_test_name = "irds:beir/scifact/test"

time_fct(
    default_complete_test_pipeline_name, dataset_name, dataset_test_name, q_encoder, eval_metrics)

100%|██████████| 5183/5183 [00:00<00:00, 1446475.32it/s]


Experiment took 30.021 seconds to execute.


,name,RR@10,nDCG@10,AP@100
0,scifact: BM25 >> gte-base-en-v1.5,0.669475,0.708775,0.664073


In [ ]:
from experiment_utils.experiments_helper import split_dev_test, default_complete_test_pipeline

dataset_name = "arguana"
dataset = pt.get_dataset("irds:beir/arguana")
topics = dataset.get_topics()

# dev_topics, test_topics = split_dev_test(topics, test_size=0.8)

time_fct(default_complete_test_pipeline, dataset_name, dataset.get_qrels(), test_topics, q_encoder,
         eval_metrics)



In [13]:
import timeit
timeit.timeit('"-".join(str(n) for n in range(100))', number=10000)


0.10299960100019234